# **Layer 1: Baseline DeiT environment**

DeiT’s baseline training script expects a teacher model name and distillation settings via CLI flags in main.py (e.g., --teacher-model, --teacher-path, --distillation-type).
GitHub
+1

So the “base environment” Layer 1 must include:

DeiT repo (cloned)

PyTorch (Colab default) + GPU

timm installed (for both student and teacher models)

compatibility patches if any (because Colab uses new torch/timm)

Install PyTorch without pinning

In [1]:
!pip -q install --upgrade pip
!pip -q install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 87.7 MB/s eta 0:00:00


Verify

In [2]:
import torch
print(torch.__version__)
print("CUDA:", torch.cuda.is_available())

2.9.0+cu126
CUDA: True


Clone the baseline repo (official DeiT)

In [3]:
%cd /content
!git clone https://github.com/facebookresearch/deit.git
%cd /content/deit
!grep -n "torch" -n requirements.txt || true

/content
Cloning into 'deit'...
remote: Enumerating objects: 456, done.
remote: Total 456 (delta 0), reused 0 (delta 0), pack-reused 456 (from 1)
Receiving objects: 100% (456/456), 5.73 MiB | 22.75 MiB/s, done.
Resolving deltas: 100% (255/255), done.
/content/deit
1:torch==1.13.1
2:torchvision==0.8.1


Colab Compatibility Fixes

1. torch pin removal

2. timm API changes

3. kwargs popping (pretrained_cfg, cache_dir, etc.)



Patch requirements.txt to remove torch pins

In [4]:
%cd /content/deit

!python - << 'PY'
from pathlib import Path
p = Path("requirements.txt")
lines = p.read_text().splitlines()

filtered = []
removed = []
for line in lines:
    s = line.strip()
    if s.startswith("torch==") or s.startswith("torchvision==") or s.startswith("torchaudio=="):
        removed.append(line)
        continue
    filtered.append(line)

p.write_text("\n".join(filtered) + "\n")
print("✅ Removed these pinned lines:")
for r in removed:
    print("  -", r)

/content/deit
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
✅ Removed these pinned lines:
  - torch==1.13.1
  - torchvision==0.8.1


Verify Pins are gone!i.e torch==1.13.1 pin was removed

In [5]:
!grep -nE "torch|torchvision|torchaudio" requirements.txt || echo "✅ No torch pins remain"

✅ No torch pins remain


Install the baseline dependencies

In [6]:
pip install "jedi>=0.16,<0.19"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.9 MB/s  0:00:00


In [7]:
!pip -q uninstall -y timm
!pip -q install "jedi>=0.16,<0.19"
# !pip -q install timm==0.6.13 submitit
!pip -q install timm==0.4.12 submitit


In [ ]:
#!pip -q uninstall -y timm
#!pip -q install -U pip setuptools wheel
#!pip -q install -U "timm>=1.0.0"

Verify

In [8]:
!python -c "import timm; print('timm:', timm.__version__)"
#0.4.12

timm: 0.4.12


**Restart the Session**

In [1]:
!python - << 'PY'
from pathlib import Path

p = Path("/usr/local/lib/python3.12/dist-packages/timm/data/__init__.py")
txt = p.read_text()

needle = "OPENAI_CLIP_MEAN"
if needle in txt:
    print("✅ timm.data already mentions OPENAI_CLIP_MEAN; no patch needed.")
else:
    patch = """

# --- Colab patch: expose CLIP normalization constants for older exports ---
try:
    from .constants import OPENAI_CLIP_MEAN, OPENAI_CLIP_STD  # timm versions where defined in constants
except Exception:
    # Standard OpenAI CLIP normalization
    OPENAI_CLIP_MEAN = (0.48145466, 0.4578275, 0.40821073)
    OPENAI_CLIP_STD  = (0.26862954, 0.26130258, 0.27577711)
# --- end patch ---
"""
    p.write_text(txt + patch)
    print("✅ Patched:", p)

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
✅ timm.data already mentions OPENAI_CLIP_MEAN; no patch needed.


Runtime → Restart runtime

In [ ]:
#!pip -q install timm submitit

In [2]:
%cd /content/deit
from models import deit_tiny_patch16_224
m = deit_tiny_patch16_224()
print("✅ DeiT model instantiated successfully")

/content/deit
✅ DeiT model instantiated successfully


In [3]:
import torch, timm
print(torch.__version__)
print(timm.__version__)
print(torch.cuda.is_available())

2.9.0+cu126
0.4.12
True


Download Tiny-ImageNet

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content
!wget -q http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

/content


Fix Tiny-ImageNet validation folder

In [4]:
!python - << 'EOF'
import shutil
from pathlib import Path

root = Path("/content/tiny-imagenet-200")
val_dir = root/"val"
img_dir = val_dir/"images"
ann = val_dir/"val_annotations.txt"

with ann.open("r") as f:
    for line in f:
        img, cls = line.strip().split("\t")[:2]
        (val_dir/cls).mkdir(parents=True, exist_ok=True)
        src = img_dir/img
        dst = val_dir/cls/img
        if src.exists():
            shutil.move(str(src), str(dst))

if img_dir.exists():
    shutil.rmtree(img_dir)

print("✅ Tiny-ImageNet val reorganized into class subfolders.")

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')
✅ Tiny-ImageNet val reorganized into class subfolders.


In [5]:
!find /content/tiny-imagenet-200/val -maxdepth 1 -type d | head

/content/tiny-imagenet-200/val
/content/tiny-imagenet-200/val/n03891332
/content/tiny-imagenet-200/val/n04275548
/content/tiny-imagenet-200/val/n07695742
/content/tiny-imagenet-200/val/n01950731
/content/tiny-imagenet-200/val/n01770393
/content/tiny-imagenet-200/val/n03393912
/content/tiny-imagenet-200/val/n03179701
/content/tiny-imagenet-200/val/n09256479
/content/tiny-imagenet-200/val/n04070727


In [6]:
ls -lah /content/tiny-imagenet-200 | head

total 2.6M
drwxrwxr-x   5 root root 4.0K Feb  9  2015 ./
drwxr-xr-x   1 root root 4.0K Jan 25 23:11 ../
drwxrwxr-x   3 root root 4.0K Dec 12  2014 test/
drwxrwxr-x 202 root root 4.0K Dec 12  2014 train/
drwxrwxr-x 202 root root 4.0K Jan 25 22:50 val/
-rw-rw-r--   1 root root 2.0K Feb  9  2015 wnids.txt
-rw-------   1 root root 2.6M Feb  9  2015 words.txt


Handle timm incompatibilities. Although we can instantiate the model directly, the training script uses timm.create_model(), which injects metadata arguments such as pretrained_cfg and cache_dir.
The original DeiT constructors do not support these arguments, so we remove them
YOUR NOTEBOOK CALL
    |
    v
deit_tiny_patch16_224()          ✅ works (no kwargs)

TRAINING PIPELINE
    |
    v
timm.create_model()
    |
    v
deit_tiny_patch16_224(**kwargs)  ❌ injects extra keys


Patch /content/deit/augment.py (safe compatibility fix)

In [7]:
%cd /content/deit
!python - << 'PY'
from pathlib import Path
p = Path("augment.py")
txt = p.read_text()

old = "from timm.data.transforms import _pil_interp, RandomResizedCropAndInterpolation, ToNumpy, ToTensor"
if old in txt:
    txt = txt.replace(
        old,
        "from timm.data.transforms import RandomResizedCropAndInterpolation, ToNumpy, ToTensor\n"
        "try:\n"
        "    from timm.data.transforms import _pil_interp  # older timm\n"
        "except Exception:\n"
        "    _pil_interp = None  # newer timm doesn't expose this\n"
    )
    p.write_text(txt)
    print("✅ Patched augment.py for timm compatibility.")
else:
    print("ℹ️ Expected import line not found; augment.py may already be patched or different.")

/content/deit
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
ℹ️ Expected import line not found; augment.py may already be patched or different.


In [8]:
%cd /content/deit
!sed -n '1,200p' models.py

/content/deit
# Copyright (c) 2015-present, Facebook, Inc.
# All rights reserved.
import torch
import torch.nn as nn
from functools import partial

from timm.models.vision_transformer import VisionTransformer, _cfg
from timm.models.registry import register_model
from timm.models.layers import trunc_normal_


__all__ = [
    'deit_tiny_patch16_224', 'deit_small_patch16_224', 'deit_base_patch16_224',
    'deit_tiny_distilled_patch16_224', 'deit_small_distilled_patch16_224',
    'deit_base_distilled_patch16_224', 'deit_base_patch16_384',
    'deit_base_distilled_patch16_384',
]


class DistilledVisionTransformer(VisionTransformer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dist_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim))
        num_patches = self.patch_embed.num_patches
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 2, self.embed_dim))
        self.head_dist = nn.Linear(self.embed_dim, self.num_classes)

In [9]:
%cd /content/deit
!rm -f multiteacher_loss.py
!ls -l multiteacher_loss.py || echo "✅ old file removed"

/content/deit
ls: cannot access 'multiteacher_loss.py': No such file or directory
✅ old file removed


In [10]:
%cd /content/deit

from pathlib import Path

code = r'''
from __future__ import annotations

from typing import Dict, List, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
import timm


# ---------------------------
# Utilities
# ---------------------------
def normalize_lambdas(lmb: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """
    Normalize teacher weights along the last dimension.
    lmb: (T,) or (B,T)
    """
    if lmb.dim() == 1:
        return lmb / lmb.sum().clamp_min(eps)
    return lmb / lmb.sum(dim=-1, keepdim=True).clamp_min(eps)


def fuse_logits(
    teacher_logits: Dict[str, torch.Tensor],
    teacher_order: List[str],
    lambdas: torch.Tensor,
) -> torch.Tensor:
    """
    teacher_logits: dict{name -> (B,C)}
    teacher_order: stable list of teacher names
    lambdas: (B,T) or (T,)
    returns: (B,C)
    """
    logits_list = [teacher_logits[k] for k in teacher_order]
    stacked = torch.stack(logits_list, dim=1)  # (B,T,C)

    lambdas = normalize_lambdas(lambdas).to(stacked.device)
    if lambdas.dim() == 1:
        lambdas = lambdas.unsqueeze(0).expand(stacked.size(0), -1)  # (B,T)

    return (stacked * lambdas.unsqueeze(-1)).sum(dim=1)  # (B,C)


def kd_soft(student_logits: torch.Tensor, teacher_logits: torch.Tensor, T: float) -> torch.Tensor:
    """
    KL( teacher || student ) with temperature scaling.
    """
    T = float(T)
    teacher_logits = teacher_logits.to(student_logits.device)
    p_t = F.softmax(teacher_logits / T, dim=-1)
    log_p_s = F.log_softmax(student_logits / T, dim=-1)
    return F.kl_div(log_p_s, p_t, reduction="batchmean") * (T * T)


def kd_hard(student_logits: torch.Tensor, teacher_logits: torch.Tensor) -> torch.Tensor:
    """
    Hard distillation: use teacher argmax labels.
    """
    y_t = teacher_logits.argmax(dim=-1)
    return F.cross_entropy(student_logits, y_t)


# ---------------------------
# Teachers
# ---------------------------
class FrozenTeacherEnsemble(nn.Module):
    """
    Loads timm teachers (ImageNet pretrained), freezes them, and returns logits dict.
    """
    def __init__(self, teacher_names: List[str], device: torch.device):
        super().__init__()
        self.device = device
        models = {}
        for name in teacher_names:
            m = timm.create_model(name, pretrained=True, num_classes=1000)
            m.eval()
            for p in m.parameters():
                p.requires_grad_(False)
            models[name] = m.to(device)
        self.models = nn.ModuleDict(models)
        self.teacher_order = list(self.models.keys())

    @torch.no_grad()
    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        return {k: m(x) for k, m in self.models.items()}


class TeacherLogitAdapter(nn.Module):
    """
    Maps teacher logits (1000) -> student_num_classes (e.g., 200 for Tiny-ImageNet).
    This adapter MUST be optimized in main.py by adding its params to optimizer.
    """
    def __init__(self, teacher_keys: List[str], student_num_classes: int):
        super().__init__()
        self.adapters = nn.ModuleDict({
            k: nn.Linear(1000, student_num_classes, bias=False) for k in teacher_keys
        })

    def forward(self, teacher_logits: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        return {k: self.adapters[k](v) for k, v in teacher_logits.items()}


# ---------------------------
# HDTSE (mixup-safe)
# ---------------------------
class HDTSEConfidence(nn.Module):
    """
    Dynamic teacher weights based on teacher confidence on the target.
    Works for:
      - hard labels: targets shape (B,)
      - mixup/cutmix soft labels: targets shape (B,C)
    """
    def __init__(self, temp: float = 1.0):
        super().__init__()
        self.temp = float(temp)

    @torch.no_grad()
    def forward(
        self,
        student_logits: torch.Tensor,  # unused, but kept for extensibility
        teacher_logits: Dict[str, torch.Tensor],
        teacher_order: List[str],
        targets: torch.Tensor,
    ) -> torch.Tensor:
        logits_list = [teacher_logits[k] for k in teacher_order]
        stacked = torch.stack(logits_list, dim=1)  # (B,T,C)
        probs = F.softmax(stacked / self.temp, dim=-1)  # (B,T,C)

        # Case A: hard labels (B,)
        if targets.dim() == 1:
            idx = targets.to(dtype=torch.long, device=probs.device)
            conf = probs.gather(-1, idx[:, None, None]).squeeze(-1)  # (B,T)
            return normalize_lambdas(conf)

        # Case B: soft labels from mixup/cutmix (B,C)
        tgt = targets.to(dtype=probs.dtype, device=probs.device)  # (B,C)
        conf = (probs * tgt[:, None, :]).sum(dim=-1)  # (B,T)
        return normalize_lambdas(conf)


# ---------------------------
# Multi-teacher distillation loss
# ---------------------------
class MultiTeacherDistillationLoss(nn.Module):
    """
    Called from engine as:
        loss = criterion(samples, outputs, targets)

    - base_criterion: CE / SoftTargetCE etc (from timm)
    - distillation_type: "soft" or "hard"
    - alpha: weight for KD
    - tau: temperature for KD
    """
    def __init__(
        self,
        base_criterion: nn.Module,
        student_num_classes: int,
        teacher_names: List[str],
        distillation_type: str = "soft",
        alpha: float = 0.5,
        tau: float = 2.0,
        device: Optional[torch.device] = None,
        use_adapter: bool = True,
    ):
        super().__init__()
        self.base_criterion = base_criterion
        self.distillation_type = distillation_type
        self.alpha = float(alpha)
        self.tau = float(tau)
        self.device = device if device is not None else torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.teachers = FrozenTeacherEnsemble(teacher_names, device=self.device)
        self.teacher_order = self.teachers.teacher_order

        self.adapter = TeacherLogitAdapter(self.teacher_order, student_num_classes).to(self.device) if use_adapter else None
        self.hdtse = HDTSEConfidence(temp=1.0)

        # Optional: store last batch lambdas for debugging/logging
        self.last_lambdas: Optional[torch.Tensor] = None  # (B,T)

    def forward(self, inputs: torch.Tensor, outputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        # Base loss (handles hard or soft labels depending on what base_criterion is)
        base_loss = self.base_criterion(outputs, targets)

        # Teacher forward (frozen)
        with torch.no_grad():
            t_logits = self.teachers(inputs)  # dict of (B,1000)

        # Adapt to student class space
        if self.adapter is not None:
            t_logits = self.adapter(t_logits)  # dict of (B,C_student)

        # Dynamic weights (mixup-safe)
        order = self.teacher_order
        with torch.no_grad():
            lambdas = self.hdtse(outputs.detach(), t_logits, order, targets)  # (B,T)
        self.last_lambdas = lambdas

        # Fuse teacher logits
        fused = fuse_logits(t_logits, order, lambdas)  # (B,C_student)

        # KD loss
        if self.distillation_type == "hard":
            distill_loss = kd_hard(outputs, fused)
        else:
            distill_loss = kd_soft(outputs, fused, self.tau)

        # Final loss
        return (1.0 - self.alpha) * base_loss + self.alpha * distill_loss
'''

path = Path("multiteacher_loss.py")
path.write_text(code)

print("File written:", path)
print("File size (bytes):", path.stat().st_size)

/content/deit
File written: multiteacher_loss.py
File size (bytes): 7288


In [11]:
from pathlib import Path
import re, shutil
from datetime import datetime
import py_compile

MAIN = Path("/content/deit/main.py")
assert MAIN.exists(), f"main.py not found at {MAIN}"

# -----------------------
# Backup
# -----------------------
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
bak = MAIN.with_name(f"main.py.bak_finalpatch_{ts}")
shutil.copy2(MAIN, bak)
print("✅ Backup created:", bak)

txt = MAIN.read_text()

# -----------------------
# Patch A: Ensure import exists
# -----------------------
if "from multiteacher_loss import MultiTeacherDistillationLoss" not in txt:
    if "from losses import DistillationLoss" in txt:
        txt = txt.replace(
            "from losses import DistillationLoss",
            "from losses import DistillationLoss\nfrom multiteacher_loss import MultiTeacherDistillationLoss",
            1,
        )
        print("✅ Added MultiTeacherDistillationLoss import")
    else:
        # fallback insertion near end of imports
        txt = txt.replace(
            "import utils",
            "import utils\nfrom multiteacher_loss import MultiTeacherDistillationLoss",
            1,
        )
        print("✅ Added MultiTeacherDistillationLoss import (fallback)")

# -----------------------
# Patch B: Add --teacher-models CLI arg (after --teacher-path)
# -----------------------
if "--teacher-models" not in txt:
    anchor = "parser.add_argument('--teacher-path', type=str, default='')"
    addition = (
        "    parser.add_argument('--teacher-models', type=str, default='',\n"
        "                        help='Comma-separated timm model names for multi-teacher distillation')\n"
    )
    if anchor in txt:
        txt = txt.replace(anchor, anchor + "\n" + addition, 1)
        print("✅ Added --teacher-models argument")
    else:
        raise RuntimeError("❌ Could not find '--teacher-path' to insert '--teacher-models' after it.")

# -----------------------
# Patch C: Relax teacher-path assertion (teacher-path OR teacher-models)
# -----------------------
# Replace strict assertion if present
txt, n_assert = re.subn(
    r"assert\s+args\.teacher_path\s*,\s*['\"]need to specify teacher-path when using distillation['\"]",
    "assert (args.teacher_path or args.teacher_models), 'need to specify teacher-path OR teacher-models when using distillation'",
    txt,
)
if n_assert:
    print(f"✅ Relaxed teacher-path assertion ({n_assert} replacements)")

# -----------------------
# Patch D: Allow finetune + MULTI-teacher distillation (block only single-teacher)
# -----------------------
finetune_pat = re.compile(
    r"if\s+args\.distillation_type\s*!=\s*'none'\s+and\s+args\.finetune\s+and\s+not\s+args\.eval:\s*\n\s*raise\s+NotImplementedError\([\"']Finetuning with distillation not yet supported[\"']\)",
    re.MULTILINE,
)
if finetune_pat.search(txt):
    txt = finetune_pat.sub(
        "if args.distillation_type != 'none' and args.finetune and not args.eval and not args.teacher_models:\n"
        "        raise NotImplementedError(\"Finetuning with single-teacher distillation not yet supported\")",
        txt,
        count=1,
    )
    print("✅ Patched finetune+distill guard (multi-teacher allowed)")

# -----------------------
# Patch E: Ensure multi-teacher branch prints confirmation
# (print teachers list right after parsing teacher_names)
# -----------------------
# Insert after teacher_names assignment if not already present
if "Multi-teacher distillation enabled. Teachers:" not in txt:
    pat_teacher_names = re.compile(
        r"(teacher_names\s*=\s*\[t\.strip\(\)\s*for\s*t\s*in\s*args\.teacher_models\.split\(','\)\s*if\s*t\.strip\(\)\]\s*\n)",
        re.MULTILINE,
    )
    ins = (
        r"\1"
        "            print(\"✅ Multi-teacher distillation enabled. Teachers:\", teacher_names)\n"
    )
    txt2, n = pat_teacher_names.subn(ins, txt, count=1)
    if n == 1:
        txt = txt2
        print("✅ Added multi-teacher confirmation print")
    else:
        print("ℹ️ Could not auto-insert teacher confirmation print (teacher_names line not matched).")

# -----------------------
# Patch F: Insert adapter->optimizer hook after MultiTeacherDistillationLoss(...)
# -----------------------
if "Added adapter parameters to optimizer" not in txt:
    mt_pat = re.compile(
        r"(criterion\s*=\s*MultiTeacherDistillationLoss\([\s\S]*?\)\n)",
        re.MULTILINE,
    )
    adapter_block = (
        "\n"
        "            # ✅ IMPORTANT: train the adapter (otherwise KD hurts accuracy)\n"
        "            if hasattr(criterion, \"adapter\") and criterion.adapter is not None:\n"
        "                optimizer.add_param_group({\n"
        "                    \"params\": criterion.adapter.parameters(),\n"
        "                    \"lr\": args.lr,\n"
        "                    \"weight_decay\": 0.0,\n"
        "                })\n"
        "                print(\"✅ Added adapter parameters to optimizer\")\n"
    )
    txt2, n = mt_pat.subn(r"\1" + adapter_block, txt, count=1)
    if n == 1:
        txt = txt2
        print("✅ Inserted adapter->optimizer hook after MultiTeacherDistillationLoss(...)")
    else:
        print("ℹ️ Could not insert adapter hook automatically (MultiTeacherDistillationLoss block not found uniquely).")

# -----------------------
# Patch 7 (CRITICAL): Fix scheduler creation safely
#   - remove ALL existing create_scheduler lines anywhere
#   - insert exactly once BEFORE 'output_dir = Path(args.output_dir)' with 4-space indent
# -----------------------
txt = re.sub(r"^\s*lr_scheduler\s*,\s*_\s*=\s*create_scheduler\(\s*args\s*,\s*optimizer\s*\)\s*$", "", txt, flags=re.MULTILINE)

insert_anchor = "    output_dir = Path(args.output_dir)"
if insert_anchor not in txt:
    raise RuntimeError("❌ Could not find 'output_dir = Path(args.output_dir)' anchor to insert scheduler safely.")

if "lr_scheduler, _ = create_scheduler(args, optimizer)" not in txt:
    txt = txt.replace(
        insert_anchor,
        "    lr_scheduler, _ = create_scheduler(args, optimizer)\n\n" + insert_anchor,
        1,
    )
    print("✅ Inserted scheduler creation safely before output_dir")

# Clean up excessive blank lines caused by removals
txt = re.sub(r"\n{3,}", "\n\n", txt)

# -----------------------
# Write + compile check
# -----------------------
MAIN.write_text(txt)
print("✅ Wrote patched main.py")

py_compile.compile(str(MAIN), doraise=True)
print("✅ main.py compiles successfully")


✅ Backup created: /content/deit/main.py.bak_finalpatch_20260126_001359
✅ Inserted scheduler creation safely before output_dir
✅ Wrote patched main.py
✅ main.py compiles successfully


Before constructing the model, remove those keys from kwargs

In [12]:
from pathlib import Path

p = Path("/content/deit/models.py")
lines = p.read_text().splitlines()

out = []
for line in lines:
    out.append(line)
    if line.strip().startswith("def deit_") and "**kwargs" in line:
        out.append("    # Drop timm-injected kwargs not supported by DeiT")
        out.append("    kwargs.pop('pretrained_cfg', None)")
        out.append("    kwargs.pop('pretrained_cfg_overlay', None)")
        out.append("    kwargs.pop('pretrained_cfg_priority', None)")

p.write_text("\n".join(out) + "\n")
print("✅ models.py patched to drop pretrained_cfg kwargs")


✅ models.py patched to drop pretrained_cfg kwargs


Verify

In [13]:
!grep -n "pretrained_cfg" /content/deit/models.py

65:    kwargs.pop('pretrained_cfg', None)
66:    kwargs.pop('pretrained_cfg_overlay', None)
67:    kwargs.pop('pretrained_cfg_priority', None)
73:    kwargs.pop('pretrained_cfg', None)
74:    kwargs.pop('pretrained_cfg_overlay', None)
75:    kwargs.pop('pretrained_cfg_priority', None)
92:    kwargs.pop('pretrained_cfg', None)
93:    kwargs.pop('pretrained_cfg_overlay', None)
94:    kwargs.pop('pretrained_cfg_priority', None)
100:    kwargs.pop('pretrained_cfg', None)
101:    kwargs.pop('pretrained_cfg_overlay', None)
102:    kwargs.pop('pretrained_cfg_priority', None)
119:    kwargs.pop('pretrained_cfg', None)
120:    kwargs.pop('pretrained_cfg_overlay', None)
121:    kwargs.pop('pretrained_cfg_priority', None)
127:    kwargs.pop('pretrained_cfg', None)
128:    kwargs.pop('pretrained_cfg_overlay', None)
129:    kwargs.pop('pretrained_cfg_priority', None)
146:    kwargs.pop('pretrained_cfg', None)
147:    kwargs.pop('pretrained_cfg_overlay', None)
148:    kwargs.pop('pretrained_cfg_prio

Fix: Patch /content/deit/models.py to drop pretrained_cfg=...

Patch models.py to also drop cache_dir (and friends)

In [14]:
from pathlib import Path

p = Path("/content/deit/models.py")
lines = p.read_text().splitlines()

# Keys that timm may inject but DeiT constructors don't accept
DROP_KEYS = [
    "cache_dir",
    "hf_hub_id",
    "hf_hub_filename",
    "hf_hub_revision",
]

out = []
for line in lines:
    out.append(line)
    # Right after the comment line we previously inserted, add more pops once per function
    if line.strip() == "# Drop timm-injected kwargs not supported by DeiT":
        for k in DROP_KEYS:
            out.append(f"    kwargs.pop('{k}', None)")

p.write_text("\n".join(out) + "\n")
print("✅ Patched models.py to drop cache_dir/hf_hub* kwargs")


✅ Patched models.py to drop cache_dir/hf_hub* kwargs


Verify

In [15]:
!grep -n "cache_dir" /content/deit/models.py

65:    kwargs.pop('cache_dir', None)
73:    kwargs.pop('cache_dir', None)
77:    kwargs.pop('cache_dir', None)
100:    kwargs.pop('cache_dir', None)
108:    kwargs.pop('cache_dir', None)
112:    kwargs.pop('cache_dir', None)
135:    kwargs.pop('cache_dir', None)
143:    kwargs.pop('cache_dir', None)
147:    kwargs.pop('cache_dir', None)
170:    kwargs.pop('cache_dir', None)
178:    kwargs.pop('cache_dir', None)
182:    kwargs.pop('cache_dir', None)
205:    kwargs.pop('cache_dir', None)
213:    kwargs.pop('cache_dir', None)
217:    kwargs.pop('cache_dir', None)
240:    kwargs.pop('cache_dir', None)
248:    kwargs.pop('cache_dir', None)
252:    kwargs.pop('cache_dir', None)
275:    kwargs.pop('cache_dir', None)
283:    kwargs.pop('cache_dir', None)
287:    kwargs.pop('cache_dir', None)
310:    kwargs.pop('cache_dir', None)
318:    kwargs.pop('cache_dir', None)
322:    kwargs.pop('cache_dir', None)


In [16]:
# %cd /content/deit
# !python main.py \
#   --model deit_tiny_patch16_224 \
#   --data-path /content/tiny-imagenet-200 \
#   --pretrained \
#   --epochs 1 \
#   --batch-size 64 \
#   --num_workers 2 \
#   --output_dir /content/deit_runs/smoke_test
# %cd /content/deit
# !python main.py \
#   --model deit_tiny_patch16_224 \
#   --data-path /content/tiny-imagenet-200 \
#   --epochs 1 \
#   --batch-size 128 \
#   --num_workers 4 \
#   --input-size 224 \
#   --opt adamw \
#   --lr 5e-4 \
#   --weight-decay 0.05 \
#   --sched cosine \
#   --aa rand-m9-mstd0.5 \
#   --reprob 0.25 \
#   --remode pixel \
#   --recount 1 \
#   --output_dir /content/deit_runs/tiny_imagenet
### correct one
# %cd /content/deit
# !python main.py \
#  --model deit_tiny_patch16_224 \
#  --data-path /content/tiny-imagenet-200 \
#  --finetune https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth \
#  --epochs 10 \
#  --batch-size 128 \
#  --num_workers 4 \
#  --input-size 224 \
#  --opt adamw \
#  --lr 3e-4 \
#  --weight-decay 0.05 \
#  --sched cosine \
#  --warmup-epochs 1 \
#  --smoothing 0.1 \
#  --aa rand-m7-mstd0.5 \
#  --reprob 0.1 \
#  --drop-path 0.1 \
#  --output_dir /content/deit_runs/tiny_imagenet_5ep
%cd /content/deit
!python main.py \
 --model deit_tiny_patch16_224 \
 --data-path /content/tiny-imagenet-200 \
 --finetune https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth \
 --epochs 10 \
 --batch-size 128 \
 --num_workers 4 \
 --input-size 224 \
 --opt adamw \
 --lr 2.5e-4 \
 --weight-decay 0.05 \
 --sched cosine \
 --warmup-epochs 0 \
 --smoothing 0.1 \
 --aa rand-m6-mstd0.5 \
 --reprob 0.1 \
 --drop-path 0.05 \
 --mixup 0.2 \
 --cutmix 0.0 \
 --mixup-prob 0.5 \
 --distillation-type soft \
 --distillation-alpha 0.2 \
 --distillation-tau 2.0 \
 --output_dir /content/deit_runs/tiny_imagenet \
 --teacher-models "tf_efficientnet_b0,tf_efficientnet_b1,mobilenetv3_large_100"
# %cd /content/deit
# !python main.py \
#  --model deit_tiny_patch16_224 \
#  --data-path /content/tiny-imagenet-200 \
#  --finetune https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth \
#  --epochs 10 \
#  --batch-size 128 \
#  --num_workers 4 \
#  --input-size 224 \
#  --opt adamw \
#  --lr 2.5e-4 \
#  --weight-decay 0.05 \
#  --sched cosine \
#  --warmup-epochs 1 \
#  --smoothing 0.1 \
#  --aa rand-m7-mstd0.5 \
#  --reprob 0.1 \
#  --drop-path 0.1 \
#  --distillation-type hard \
# --teacher-model regnety_160 \
# --teacher-path https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth \
#  --output_dir /content/deit_runs/tiny_imagenet_10ep
# %cd /content/deit
# !python main.py \
#  --model deit_tiny_distilled_patch16_224 \
#  --data-path /content/tiny-imagenet-200 \
#  --epochs 10 \
#  --batch-size 128 \
#  --num_workers 4 \
#  --input-size 224 \
#  --opt adamw \
#  --lr 7e-4 \
#  --weight-decay 0.05 \
#  --sched cosine \
#  --warmup-epochs 1 \
#  --smoothing 0.0 \
#  --aa rand-m7-mstd0.5 \
#  --reprob 0.1 \
#  --drop-path 0.0 \
#  --distillation-type hard \
#  --distillation-alpha 0.7 \
#  --teacher-model regnety_160 \
#  --teacher-path https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth \
#  --output_dir /content/deit_runs/deit_tiny_distilled_10ep





/content/deit
Not using distributed mode
Namespace(batch_size=128, epochs=10, bce_loss=False, unscale_lr=False, model='deit_tiny_patch16_224', input_size=224, drop=0.0, drop_path=0.05, model_ema=True, model_ema_decay=0.99996, model_ema_force_cpu=False, opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=None, momentum=0.9, weight_decay=0.05, sched='cosine', lr=0.00025, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, warmup_lr=1e-06, min_lr=1e-05, decay_epochs=30, warmup_epochs=0, cooldown_epochs=10, patience_epochs=10, decay_rate=0.1, color_jitter=0.3, aa='rand-m6-mstd0.5', smoothing=0.1, train_interpolation='bicubic', repeated_aug=True, train_mode=True, ThreeAugment=False, src=False, reprob=0.1, remode='pixel', recount=1, resplit=False, mixup=0.2, cutmix=0.0, cutmix_minmax=None, mixup_prob=0.5, mixup_switch_prob=0.5, mixup_mode='batch', teacher_model='regnety_160', teacher_path='', teacher_models='tf_efficientnet_b0,tf_efficientnet_b1,mobilenetv3_large_100', distillation_type='s

# **Layer 2: Base Environment — Teacher Models & Multi-Teacher Adaptations**

Layer 2 extends the baseline DeiT environment to support knowledge distillation from one or more teacher models. This layer is additive: it does not modify the baseline DeiT training loop unless explicitly stated.
It includes
1. Teacher Model Support (Single & Multiple)
2. Teacher Registry / Configuration
3. Multi-Teacher Fusion Mechanism (Adaptation Layer)
4. Distillation Loss Integration